# Portfolio Factor Regressions

Let's run some regressions

## 1. Get return data for securities (using closing prices)

In [1]:
# Import packages
import requests
from datetime import datetime
import pandas as pd

In [2]:
# API Formatting
api_url = "https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY_ADJUSTED&symbol={}&apikey="
api_key = ""
api = api_url + api_key 

In [3]:
# Define closing_price function to retrieve closing price for monthly data
def closing_price(data1, time_key_string):
    '''
    data1: json file from AlphaVantage
    time_key_string: The string key inputed in json file to select the time (daily, weekly, monthly)
    time_key_string2: If the security uses a different key string (e.g. includes volume)
    
    This function will create a list of list with dates and corresponding closing prices, adjusted for
    dividends (by adding it)
    '''
    date_list = list(data1[time_key_string].keys())
    closing_price_list = []
    datetime_list = []
    
    for i in date_list:
        closing_price = float(data1[time_key_string][i]['4. close']) + float(data1[time_key_string][i]['7. dividend amount'])
        closing_price_list.append(closing_price)
        datetime_list.append(datetime.strptime(i, '%Y-%m-%d').strftime('%Y-%m'))
        
    df_out = pd.DataFrame(list(zip(datetime_list, closing_price_list)), columns=['date', 'closing_price'])
    df_out = df_out.set_index('date')
    df_out = df_out.sort_index(ascending=True)
    return df_out

In [4]:
# Define returns function to calculate monthly returns from closing prices
def returns(df):
    '''
    df = dataframe with prices against time
    column_name = name of the price column used to calculate returns
    
    This function will generate the returns df
    '''
    pct_change_series = df['closing_price'].pct_change(periods=1)
    df2 = pd.DataFrame(pct_change_series)
    df2 = df2.rename(columns={'closing_price':'returns'})
    return df2

In [5]:
def get_all_securities(ticker_list, time_key_string):
    '''
    This function simply loops through and applies closing_price()
    and returns() for a list of tickers
    '''
    init = 0
    for i in ticker_list:
        url = api.format(i)
        data = requests.get(url).json()
        df_closing = closing_price(data, time_key_string)
        df_returns = returns(df_closing)
        df_returns = df_returns.rename(columns={'returns': i})
        if init == 0:
            output = df_returns
            init += 1
        else:
            output = pd.merge(output, df_returns, left_index=True, right_index=True)
            init += 1
    return output

Careful, API limited to 5 calls per minute and 500 per day

In [6]:
tickers1 = ['VUG', 'VWO', 'BNDX']
tickers2 = ['PYPL', 'BABA', 'BLV']

In [7]:
output1 = get_all_securities(tickers1,'Monthly Adjusted Time Series')

In [8]:
output2 = get_all_securities(tickers2, 'Monthly Adjusted Time Series')

In [9]:
final = pd.merge(output1, output2, left_index=True, right_index=True)

In [11]:
#Save csv
final.to_csv('security_returns.csv', sep=',', encoding='utf-8')

In [10]:
final.head()

,VUG,VWO,BNDX,PYPL,BABA,BLV
date,,,,,,
2015-08,-0.062958,-0.098643,-0.008126,NaN,-0.155987,-0.014326
2015-09,-0.028507,-0.028952,0.007147,-0.113143,-0.108137,0.006570
2015-10,0.085982,0.038760,0.004813,0.160116,0.421570,0.004042
2015-11,0.001464,-0.022962,0.000733,-0.020828,0.002982,-0.011720
2015-12,-0.024185,-0.034401,-0.002928,0.026659,-0.033421,-0.015014


## 2. Get (Global) Fama French Factors

In [11]:
FFC4_path = "/Users/horacefung/Desktop/International_Data/Global_3_Factors.csv"
MOM_path = "/Users/horacefung/Desktop/International_Data/Global_MOM_Factor.csv"

In [12]:
FFC4 = pd.read_csv(FFC4_path)
MOM = pd.read_csv(MOM_path)

In [13]:
FFC4['Date'] = FFC4['Date'].apply(lambda x: datetime.strptime(str(x), '%Y%m').strftime('%Y-%m'))
MOM['Date'] = MOM['Date'].apply(lambda x: datetime.strptime(str(x), '%Y%m').strftime('%Y-%m'))

In [14]:
FFC4 = FFC4.set_index('Date')
MOM = MOM.set_index('Date')

FFC4 = FFC4.iloc[:, 4:8]
MOM = pd.DataFrame(MOM['WML.1'])

In [15]:
input_data = pd.merge(final, FFC4, left_index=True, right_index=True)
input_data = pd.merge(input_data, MOM, left_index=True, right_index=True)

In [16]:
def excess_returns(data, ticker_list, rf):
    for i in ticker_list:
        data[i] = data[i] - data[rf]
    return data

In [17]:
tickers = tickers1 + tickers2

input_data = excess_returns(input_data, tickers, 'RF.1')

## 3. Create portfolio

In [18]:
import math

In [19]:
tickers

['VUG', 'VWO', 'BNDX', 'PYPL', 'BABA', 'BLV']

In [20]:
weights = [0.524, 0.049, 0.063, 0.141, 0.09, 0.133]

In [21]:
input_data['Portfolio_Return'] = input_data[tickers].mul(weights).sum(1)

In [88]:
average_excess_return = input_data['Portfolio_Return'].mean() * 12
print(average_excess_return)

0.10524803225230524


In [81]:
average_std = input_data['Mkt-RF.1'].std() * math.sqrt(12)
print(average_std)

0.11109246030600863


In [90]:
dr = input_data[input_data['Portfolio_Return'] < 0]['Portfolio_Return'].std() * math.sqrt(12)
dr

0.07832581963895462

In [24]:
print("Sharpe Ratio: " + str(average_excess_return/average_std))

Sharpe Ratio: 0.8467338884807383


In [91]:
print("Sortino Ratio: " + str(average_excess_return/dr))

Sortino Ratio: 1.343720790123224


## 4. Regression

In [42]:
input_data.columns.values

array(['VUG', 'VWO', 'BNDX', 'PYPL', 'BABA', 'BLV', 'Mkt-RF.1', 'SMB.1',
       'HML.1', 'RF.1', 'WML.1', 'Portfolio_Return'], dtype=object)

In [28]:
import statsmodels.api as sm

X = sm.add_constant(input_data[['Mkt-RF.1', 'SMB.1','HML.1', 'WML.1']])
y = input_data["Portfolio_Return"]

# Note the difference in argument order
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       Portfolio_Return   R-squared:                       0.881
Model:                            OLS   Adj. R-squared:                  0.867
Method:                 Least Squares   F-statistic:                     62.90
Date:                Tue, 25 Dec 2018   Prob (F-statistic):           3.10e-15
Time:                        18:54:14   Log-Likelihood:                 116.44
No. Observations:                  39   AIC:                            -222.9
Df Residuals:                      34   BIC:                            -214.6
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0032      0.002      1.485      0.147      -0.001       0.008
Mkt-RF.1       0.9841      0.069     14.192      0.000       0.843       1.125
SMB.1         -0.2289      0.175     -1.310      0.199      -0.584       0.126
HML.1         -0.5213      0.146     -3.574      0.001      -0.818      -0.225
WML.1          0.0204      0.118      0.172      0.864      -0.220       0.261
==============================================================================
Omnibus:                        2.524   Durbin-Watson:                   2.221
Prob(Omnibus):                  0.283   Jarque-Bera (JB):                1.909
Skew:                           0.374   Prob(JB):                        0.385
Kurtosis:                       2.216   Cond. No.                         89.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
input_data[tickers + ['Mkt-RF.1', 'SMB.1','HML.1', 'WML.1']].corr()

,VUG,VWO,BNDX,PYPL,BABA,BLV,Mkt-RF.1,SMB.1,HML.1,WML.1
VUG,1.000000,0.699208,-0.014854,0.596362,0.715504,0.271135,0.932730,0.059649,-0.348295,0.024915
VWO,0.699208,1.000000,0.065908,0.359342,0.673471,0.468263,0.789311,0.097795,-0.053166,-0.014194
BNDX,-0.014854,0.065908,1.000000,-0.112896,-0.068946,0.710273,-0.091546,-0.020908,-0.284797,0.075576
PYPL,0.596362,0.359342,-0.112896,1.000000,0.604126,0.154655,0.538323,-0.022189,-0.339186,0.135661
BABA,0.715504,0.673471,-0.068946,0.604126,1.000000,0.211208,0.727952,0.080878,-0.163587,-0.048734
BLV,0.271135,0.468263,0.710273,0.154655,0.211208,1.000000,0.232232,0.146979,-0.331569,0.159473
Mkt-RF.1,0.932730,0.789311,-0.091546,0.538323,0.727952,0.232232,1.000000,0.090589,-0.075178,-0.167479
SMB.1,0.059649,0.097795,-0.020908,-0.022189,0.080878,0.146979,0.090589,1.000000,-0.213836,0.081639
HML.1,-0.348295,-0.053166,-0.284797,-0.339186,-0.163587,-0.331569,-0.075178,-0.213836,1.000000,-0.643433
WML.1,0.024915,-0.014194,0.075576,0.135661,-0.048734,0.159473,-0.167479,0.081639,-0.643433,1.000000


## 5. Optimize 

In [25]:
opt_data = input_data.copy()

In [104]:
equity_tickers = ['VUG','VWO', 'BNDX','BLV']

In [73]:
def portfolio_return(weights, opt_data):
    returns = opt_data[equity_tickers].mul(weights).sum(1)
    returns = returns.mean() * 12
    return returns

In [74]:
def portfolio_volatility(weights, opt_data):
    returns = opt_data[equity_tickers].mul(weights).sum(1)
    returns = returns.std() * math.sqrt(12)
    return returns

In [92]:
def portfolio_downside(weights, opt_data):
    returns = opt_data[equity_tickers].mul(weights).sum(1)
    down_returns = returns[returns < 0].std() * math.sqrt(12)
    return down_returns

In [75]:
def sharpe_ratio(weights):
    global opt_data
    return -portfolio_return(weights, opt_data)/portfolio_volatility(weights, opt_data)

In [96]:
def sortino_ratio(weights):
    global opt_data
    return -portfolio_return(weights, opt_data)/portfolio_downside(weights, opt_data)

In [105]:
import numpy as np
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.optimize import Bounds

linear_constraint = LinearConstraint([1,1,1,1], [0], [1])
bounds = Bounds([0,0,0,0], [10,10,10,10])

x0 = [0.7,0.1,0.1,0.1]

res = minimize(sortino_ratio, x0, constraints=[linear_constraint], bounds=bounds)

In [106]:
list(res.x)

[0.9083229219548346, 1.290108349405947e-14, 0.0, 1.0707785682386715e-14]

In [99]:
sortino_ratio(res.x)

-3.0333368196152883

In [68]:
tickers

['VUG', 'VWO', 'BNDX', 'PYPL', 'BABA', 'BLV']